<a href="https://colab.research.google.com/github/asifahmed90/Image-capturing-API-for-deep-learning-dataset/blob/master/Download_Google_Image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install google-images-download

     |████████████████████████████████| 911kB 7.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/1f/28/ad/f56e7061e1d2a9a1affe2f9c649c2570cb9198dd24ede0bbab
Successfully built google-images-download


In [0]:
import numpy as np
import tensorflow as tf
import cv2 as cv
from pathlib import Path
from send2trash import send2trash
from google_images_download import google_images_download


# Read the graph.
with tf.gfile.GFile('frozen_inference_graph.pb', 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

In [0]:
def download_images(asana):
    response = google_images_download.googleimagesdownload()
    arguments = { "keywords":asana, "limit":50, "format":'jpg', "size":"medium", "print_urls": False,
                 "image_directory":'./'+asana.lower()+'/'}
    response.download(arguments)
    arguments['format'] = "png"
    response.download(arguments)
    return

In [0]:
def detect_persons_and_save_crops(asana, graph_def, folder):
    with tf.Session() as sess:
        # Restore session
        sess.graph.as_default()
        tf.import_graph_def(graph_def, name='')

        files = list(Path(folder).glob('*.jpg'))
        files.extend(list(Path(folder).glob('*.png')))

        for fid,fname in enumerate(files): 
            # Read and preprocess an image.
            img = cv.imread(str(fname))
            if(img is not None):
                rows = img.shape[0]
                cols = img.shape[1]
                inp = cv.resize(img, (300, 300))
                inp = inp[:, :, [2, 1, 0]]  # BGR2RGB

                # Run the model
                out = sess.run([sess.graph.get_tensor_by_name('num_detections:0'),
                                sess.graph.get_tensor_by_name('detection_scores:0'),
                                sess.graph.get_tensor_by_name('detection_boxes:0'),
                                sess.graph.get_tensor_by_name('detection_classes:0')],
                               feed_dict={'image_tensor:0': inp.reshape(1, inp.shape[0], inp.shape[1], 3)})

                # Visualize detected bounding boxes.
                num_detections = int(out[0][0])
                #print(fname, num_detections)
                for i in range(num_detections):
                    classId = int(out[3][0][i])
                    if(classId==1):
                        score = float(out[1][0][i])
                        bbox = [float(v) for v in out[2][0][i]]
                        if score > 0.3:
                            x = int(bbox[1] * cols)
                            y = int(bbox[0] * rows)
                            right = int(bbox[3] * cols)
                            bottom = int(bbox[2] * rows)
                            cv.imwrite(folder+str(fid)+'-'+str(i)+'.png', img[y:bottom, x:right])

            try:
                send2trash(str(fname))
            except: pass    
        print('cleaning '+asana+' done.')
    return

In [0]:
complete_list = [ { "english_name": "Bharadvaja’s Twist", "sanskrit_name": "Bharadvajasana I", "pose_meta": [ "Hip Opening", "Seated", "Twist" ], "difficulty": 1 }, { "english_name": "Big Toe Pose", "sanskrit_name": "Padangusthasana", "pose_meta": [ "Forward Bend", "Standing" ], "difficulty": 1 }, { "english_name": "Boat Pose", "sanskrit_name": "Paripurna Navasana", "pose_meta": [ "Core", "Seated", "Strengthening" ], "difficulty": 2 }, { "english_name": "Bound Angle Pose", "sanskrit_name": "Baddha Konasana", "pose_meta": [ "Forward Bend", "Hip Opening", "Seated" ], "difficulty": 1 }, { "english_name": "Bow Pose", "sanskrit_name": "Dhanurasana", "pose_meta": [ "Chest Opening", "Backbends" ], "difficulty": 2 }, { "english_name": "Bridge Pose", "sanskrit_name": "Setu Bandha Sarvangasana", "pose_meta": [ "Backbends" ], "difficulty": 2 }, { "english_name": "Camel Pose", "sanskrit_name": "Ustrasana", "pose_meta": [ "Chest Opening", "Backbends" ], "difficulty": 2 }, { "english_name": "Cat Pose", "sanskrit_name": "Marjaryasana", "pose_meta": [ "Core" ], "difficulty": 1 }, { "english_name": "Cat Cow Pose", "sanskrit_name": "Chakravakasana", "pose_meta": [ "Backbends" ], "difficulty": 1 }, { "english_name": "Knees, Chest and Chin Pose", "sanskrit_name": "Ashtanga Namaskara", "pose_meta": [ "Core" ], "difficulty": 1 }, { "english_name": "Chair Pose", "sanskrit_name": "Utkatasana", "pose_meta": [ "Core", "Standing", "Strengthening" ], "difficulty": 2 }, { "english_name": "Child’s Pose", "sanskrit_name": "Balasana", "pose_meta": [ "Forward Bend", "Hip Opening", "Restorative" ], "difficulty": 1 }, { "english_name": "Cobra Pose", "sanskrit_name": "Bhujangasana", "pose_meta": [ "Chest Opening", "Backbends" ], "difficulty": 1 }, { "english_name": "Corpse Pose", "sanskrit_name": "Savasana", "pose_meta": [ "Restorative" ], "difficulty": 1 }, { "english_name": "Cow Face Pose", "sanskrit_name": "Gomukhasana", "pose_meta": [ "Hip Opening", "Seated" ], "difficulty": 2 }, { "english_name": "Cow Pose", "sanskrit_name": "Bitilasana", "pose_meta": [ "Chest Opening", "Backbends" ], "difficulty": 1 }, { "english_name": "Crane (Crow) Pose", "sanskrit_name": "Bakasana", "pose_meta": [ "Arm Balance", "Core" ], "difficulty": 3 }, { "english_name": "Dolphin Plank Pose", "sanskrit_name": "Makara Adho Mukha Svanasana", "pose_meta": [ "Arm Balance", "Core", "Strengthening" ], "difficulty": 2 }, { "english_name": "Dolphin Pose", "sanskrit_name": "Ardha Pincha Mayurasana", "pose_meta": [ "Core", "Standing", "Strengthening" ], "difficulty": 2 }, { "english_name": "Downward-Facing Dog", "sanskrit_name": "Adho Mukha Svanasana", "pose_meta": [ "Forward Bend", "Standing", "Strengthening" ], "difficulty": 2 }, { "english_name": "Eagle Pose", "sanskrit_name": "Garudasana", "pose_meta": [ "Balancing", "Hip Opening", "Standing" ], "difficulty": 2 }, { "english_name": "Easy Pose", "sanskrit_name": "Sukhasana", "pose_meta": [ "Hip Opening", "Seated" ], "difficulty": 1 }, { "english_name": "Eight-Angle Pose", "sanskrit_name": "Astavakrasana", "pose_meta": [ "Arm Balance" ], "difficulty": 3 }, { "english_name": "Extended Hand-To-Big-Toe Pose", "sanskrit_name": "Utthita Hasta Padangustasana", "pose_meta": [ "Balancing", "Hip Opening", "Standing" ], "difficulty": 3 }, { "english_name": "Extended Puppy Pose", "sanskrit_name": "Uttana Shishosana", "pose_meta": [ "Forward Bend" ], "difficulty": 2 }, { "english_name": "Extended Side Angle Pose", "sanskrit_name": "Utthita Parsvakonasana", "pose_meta": [ "Standing", "Strengthening" ], "difficulty": 2 }, { "english_name": "Extended Triangle Pose", "sanskrit_name": "Utthita Trikonasana", "pose_meta": [ "Standing", "Strengthening" ], "difficulty": 1 }, { "english_name": "Feathered Peacock Pose", "sanskrit_name": "Pincha Mayurasana", "pose_meta": [ "Inversion", "Strengthening" ], "difficulty": 3 }, { "english_name": "Fire Log Pose", "sanskrit_name": "Agnistambhasana", "pose_meta": [ "Hip Opening", "Seated" ], "difficulty": 1 }, { "english_name": "Firefly Pose", "sanskrit_name": "Tittibhasana", "pose_meta": [ "Arm Balance" ], "difficulty": 3 }, { "english_name": "Fish Pose", "sanskrit_name": "Matsyasana", "pose_meta": [ "Chest Opening", "Backbends" ], "difficulty": 2 }, { "english_name": "Four-Limbed Staff Pose", "sanskrit_name": "Chaturanga Dandasana", "pose_meta": [ "Arm Balance", "Core", "Strengthening" ], "difficulty": 2 }, { "english_name": "Garland Pose", "sanskrit_name": "Malasana", "pose_meta": [ "Standing" ], "difficulty": 1 }, { "english_name": "Gate Pose", "sanskrit_name": "Parighasana", "pose_meta": [ "Standing" ], "difficulty": 1 }, { "english_name": "Half Frog Pose", "sanskrit_name": "Ardha Bhekasana", "pose_meta": [ "Chest Opening", "Backbends" ], "difficulty": 2 }, { "english_name": "Half Lord of the Fishes Pose", "sanskrit_name": "Ardha Matsyendrasana", "pose_meta": [ "Hip Opening", "Seated", "Twist" ], "difficulty": 1 }, { "english_name": "Lying Half Lord of the Fishes Pose", "sanskrit_name": "Supta Matsyendrasana", "pose_meta": [ "Hip Opening", "Restorative", "Twist" ], "difficulty": 1 }, { "english_name": "Half Moon Pose", "sanskrit_name": "Ardha Chandrasana", "pose_meta": [ "Balancing", "Standing" ], "difficulty": 2 }, { "english_name": "Handstand", "sanskrit_name": "Adho Mukha Vriksasana", "pose_meta": [ "Balancing", "Inversion", "Strengthening" ], "difficulty": 3 }, { "english_name": "Happy Baby Pose", "sanskrit_name": "Ananda Balasana", "pose_meta": [ "Core" ], "difficulty": 1 }, { "english_name": "Head-to-Knee Forward Bend", "sanskrit_name": "Janu Sirsasana", "pose_meta": [ "Forward Bend", "Seated" ], "difficulty": 1 }, { "english_name": "Hero Pose", "sanskrit_name": "Virasana", "pose_meta": [ "Seated" ], "difficulty": 2 }, { "english_name": "Heron Pose", "sanskrit_name": "Krounchasana", "pose_meta": [ "Seated" ], "difficulty": 1 }, { "english_name": "High Lunge", "sanskrit_name": "Utthita Ashwa Sanchalanasana", "pose_meta": [ "Standing" ], "difficulty": 1 }, { "english_name": "Intense Side Stretch Pose", "sanskrit_name": "Parsvottanasana", "pose_meta": [ "Forward Bend", "Standing" ], "difficulty": 2 }, { "english_name": "Legs-Up-the-Wall Pose", "sanskrit_name": "Viparita Karani", "pose_meta": [ "Restorative" ], "difficulty": 1 }, { "english_name": "Locust Pose", "sanskrit_name": "Salabhasana", "pose_meta": [ "Chest Opening", "Strengthening", "Backbends" ], "difficulty": 2 }, { "english_name": "Lord of the Dance Pose", "sanskrit_name": "Natarajasana", "pose_meta": [ "Balancing", "Chest Opening", "Standing", "Backbends" ], "difficulty": 3 }, { "english_name": "Lotus Pose", "sanskrit_name": "Padmasana", "pose_meta": [ "Seated" ], "difficulty": 2 }, { "english_name": "Low Lunge", "sanskrit_name": "Anjaneyasana", "pose_meta": [ "Standing" ], "difficulty": 2 }, { "english_name": "Marichi’s Pose", "sanskrit_name": "Marichyasana III", "pose_meta": [ "Hip Opening", "Seated", "Twist" ], "difficulty": 2 }, { "english_name": "Monkey Pose", "sanskrit_name": "Hanumanasana", "pose_meta": [ "Seated" ], "difficulty": 3 }, { "english_name": "Mountain Pose", "sanskrit_name": "Tadasana", "pose_meta": [ "Standing" ], "difficulty": 1 }, { "english_name": "Noose Pose", "sanskrit_name": "Pasasana", "pose_meta": [ "Binding", "Hip Opening", "Strengthening", "Twist" ], "difficulty": 3 }, { "english_name": "One-Legged King Pigeon Pose", "sanskrit_name": "Eka Pada Rajakapotasana", "pose_meta": [ "Hip Opening", "Backbends" ], "difficulty": 3 }, { "english_name": "One-Legged King Pigeon Pose II", "sanskrit_name": "Eka Pada Rajakapotasana II", "pose_meta": [ "Backbends" ], "difficulty": 3 }, { "english_name": "Peacock Pose", "sanskrit_name": "Mayurasana", "pose_meta": [ "Arm Balance" ], "difficulty": 1 }, { "english_name": "Pigeon Pose", "sanskrit_name": "Kapotasana", "pose_meta": [ "Backbends" ], "difficulty": 3 }, { "english_name": "Plank Pose", "sanskrit_name": "Phalakasana", "pose_meta": [ "Arm Balance", "Core", "Strengthening" ], "difficulty": 1 }, { "english_name": "Plow Pose", "sanskrit_name": "Halasana", "pose_meta": [ "Inversion" ], "difficulty": 3 }, { "english_name": "Pose Dedicated to the Sage Koundinya I", "sanskrit_name": "Eka Pada Koundinyanasana I", "pose_meta": [ "Arm Balance", "Twist" ], "difficulty": 3 }, { "english_name": "Pose Dedicated to the Sage Koundinya II", "sanskrit_name": "Eka Pada Koundinyanasana II", "pose_meta": [ "Arm Balance" ], "difficulty": 3 }, { "english_name": "Pose Dedicated to the Sage Marichi I", "sanskrit_name": "Marichyasana I", "pose_meta": [ "Binding", "Forward Bend", "Hip Opening", "Seated" ], "difficulty": 2 }, { "english_name": "Reclining Bound Angle Pose", "sanskrit_name": "Supta Baddha Konasana", "pose_meta": [ "Hip Opening", "Restorative" ], "difficulty": 1 }, { "english_name": "Reclining Hand-to-Big-Toe Pose", "sanskrit_name": "Supta Padangusthasana", "pose_meta": [ "Restorative" ], "difficulty": 2 }, { "english_name": "Reclining Hero Pose", "sanskrit_name": "Supta Virasana", "pose_meta": [ "Restorative" ], "difficulty": 2 }, { "english_name": "Revolved Head-to-Knee Pose", "sanskrit_name": "Parivrtta Janu Sirsasana", "pose_meta": [ "Seated", "Twist" ], "difficulty": 1 }, { "english_name": "Revolved Side Angle Pose", "sanskrit_name": "Parivrtta Parsvakonasana", "pose_meta": [ "Standing", "Strengthening", "Twist" ], "difficulty": 2 }, { "english_name": "Revolved Triangle Pose", "sanskrit_name": "Parivrtta Trikonasana", "pose_meta": [ "Standing", "Strengthening", "Twist" ], "difficulty": 2 }, { "english_name": "Scale Pose", "sanskrit_name": "Tolasana", "pose_meta": [ "Arm Balance", "Core" ], "difficulty": 2 }, { "english_name": "Seated Forward Bend", "sanskrit_name": "Paschimottanasana", "pose_meta": [ "Forward Bend", "Seated" ], "difficulty": 1 }, { "english_name": "Shoulder-Pressing Pose", "sanskrit_name": "Bhujapidasana", "pose_meta": [ "Arm Balance" ], "difficulty": 1 }, { "english_name": "Side Crane (Crow) Pose", "sanskrit_name": "Parsva Bakasana", "pose_meta": [ "Arm Balance" ], "difficulty": 3 }, { "english_name": "Side Plank Pose", "sanskrit_name": "Vasisthasana", "pose_meta": [ "Arm Balance", "Balancing", "Core" ], "difficulty": 3 }, { "english_name": "Side-Reclining Leg Lift", "sanskrit_name": "Anantasana", "pose_meta": [ "Balancing", "Core" ], "difficulty": 1 }, { "english_name": "Sphinx Pose", "sanskrit_name": "Salamba Bhujangasana", "pose_meta": [ "Chest Opening", "Backbends" ], "difficulty": 1 }, { "english_name": "Staff Pose", "sanskrit_name": "Dandasana", "pose_meta": [ "Seated" ], "difficulty": 1 }, { "english_name": "Standing Forward Bend", "sanskrit_name": "Uttanasana", "pose_meta": [ "Forward Bend" ], "difficulty": 2 }, { "english_name": "Standing Half Forward Bend", "sanskrit_name": "Ardha Uttanasana", "pose_meta": [ "Forward Bend", "Standing" ], "difficulty": 1 }, { "english_name": "Standing Split", "sanskrit_name": "Urdhva Prasarita Eka Padasana", "pose_meta": [ "Forward Bend", "Standing" ], "difficulty": 2 }, { "english_name": "Supported Headstand", "sanskrit_name": "Salamba Sirsasana", "pose_meta": [ "Balancing", "Inversion" ], "difficulty": 3 }, { "english_name": "Supported Shoulderstand", "sanskrit_name": "Salamba Sarvangasana", "pose_meta": [ "Balancing", "Inversion" ], "difficulty": 3 }, { "english_name": "Tree Pose", "sanskrit_name": "Vriksasana", "pose_meta": [ "Balancing", "Standing" ], "difficulty": 2 }, { "english_name": "Upward Bow (Wheel) Pose", "sanskrit_name": "Urdhva Dhanurasana", "pose_meta": [ "Chest Opening", "Strengthening", "Backbends" ], "difficulty": 3 }, { "english_name": "Upward Facing Two-Foot Staff Pose", "sanskrit_name": "Dwi Pada Viparita Dandasana", "pose_meta": [ "Chest Opening", "Backbends" ], "difficulty": 3 }, { "english_name": "Upward Plank Pose", "sanskrit_name": "Purvottanasana", "pose_meta": [ "Strengthening" ], "difficulty": 2 }, { "english_name": "Upward Salute", "sanskrit_name": "Urdhva Hastasana", "pose_meta": [ "Standing" ], "difficulty": 1 }, { "english_name": "Upward-Facing Dog Pose", "sanskrit_name": "Urdhva Mukha Svanasana", "pose_meta": [ "Chest Opening", "Backbends" ], "difficulty": 2 }, { "english_name": "Warrior I Pose", "sanskrit_name": "Virabhadrasana I", "pose_meta": [ "Standing", "Strengthening" ], "difficulty": 2 }, { "english_name": "Warrior II Pose", "sanskrit_name": "Virabhadrasana II", "pose_meta": [ "Standing", "Strengthening" ], "difficulty": 2 }, { "english_name": "Warrior III Pose", "sanskrit_name": "Virabhadrasana III", "pose_meta": [ "Balancing", "Standing", "Strengthening" ], "difficulty": 3 }, { "english_name": "Wide-Angle Seated Forward Bend", "sanskrit_name": "Upavistha Konasana", "pose_meta": [ "Forward Bend", "Hip Opening", "Seated" ], "difficulty": 3 }, { "english_name": "Wide-Legged Forward Bend", "sanskrit_name": "Prasarita Padottanasana", "pose_meta": [ "Forward Bend", "Hip Opening", "Standing", "Strengthening" ], "difficulty": 1 }, { "english_name": "Wild Thing", "sanskrit_name": "Camatkarasana", "pose_meta": [ "Arm Balance", "Chest Opening" ], "difficulty": 2 }, { "english_name": "Yogic Sleep", "sanskrit_name": "Yoganidrasana", "pose_meta": [ "Forward Bend", "Seated" ], "difficulty": 3 }, { "english_name": "Scorpion Pose", "sanskrit_name": "Vrischikasana", "pose_meta": [ "Backbends", "Inversion" ], "difficulty": 3 }, { "english_name": "Thunderbolt Pose", "sanskrit_name": "Vajrasana", "pose_meta": [ "Kneeling", "Seated" ], "difficulty": 1 }, { "english_name": "Balance Pose", "sanskrit_name": "Tulasana", "pose_meta": [ "Arm Balance" ], "difficulty": 2 }, { "english_name": "Lion Pose", "sanskrit_name": "Simhasana", "pose_meta": [ "Seated" ], "difficulty": 1 }, { "english_name": "Crocodile Pose", "sanskrit_name": "Makarasana", "pose_meta": [ "Restorative" ], "difficulty": 1 }, { "english_name": "Pendant Pose", "sanskrit_name": "Lolasana", "pose_meta": [ "Arm Balance", "Core" ], "difficulty": 3 }, { "english_name": "Tortoise Pose", "sanskrit_name": "Kurmasana", "pose_meta": [ "Forward bend", "Seated" ], "difficulty": 3 }, { "english_name": "Embryo in Womb Pose", "sanskrit_name": "Garbha Pindasana", "pose_meta": [ "Seated" ], "difficulty": 3 }, { "english_name": "Durvasasana", "sanskrit_name": "Durvasasana", "pose_meta": [ "Standing", "Balancing" ], "difficulty": 3 }, { "english_name": "Bhujapidasana", "sanskrit_name": "Bhujapidasana", "pose_meta": [ "Arm Balance", "Core" ], "difficulty": 3 }, { "english_name": "Frog Pose", "sanskrit_name": "Bhekasana", "pose_meta": [ "Chest Opening", "Backbends" ], "difficulty": 2 }, { "english_name": "Formidable Pose", "sanskrit_name": "Bhairavasana", "pose_meta": [ "Hip Opening" ], "difficulty": 2 } ]

In [9]:
for item in complete_list:
    asana = item["sanskrit_name"]
    download_images(asana)
    detect_persons_and_save_crops(asana, graph_def, 'downloads/'+asana.lower()+'/')


Item no.: 1 --> Item name = Bharadvajasana I
Evaluating...
Starting Download...
Completed Image ====> 1.bharadvajasana-II.jpg
Completed Image ====> 2.bharadvajasana-1-pose.jpg
Completed Image ====> 3.bharadvajasana-I.jpg
Completed Image ====> 4.yp_292_00954_bjk.jpg
Completed Image ====> 5.bharadvajasana-prep-1.jpg
Completed Image ====> 6.111-Bharadvajasana-1-Jack-Cuneo.jpg
Completed Image ====> 7.zuckerman_kate_bharadvajasana.jpg
Completed Image ====> 8.Bharadvajasana.jpg
Completed Image ====> 9.520bb31677ae5.jpg
Completed Image ====> 10.520bb39925f94.jpg
Completed Image ====> 11.111-bharadvajasana-1-and-bharadvajasana-2-iyengar.jpg
Completed Image ====> 12.bharadvajasana-i-twist-yoga-pose-_4.jpg
Completed Image ====> 13.hqdefault.jpg
Completed Image ====> 14.112-Bharadvajasana-2-Jack-Cuneo.jpg
Completed Image ====> 15.Twist-Dedicated-to-the-Sage-Bharadvaja-I-Bharadvajasana-I-.jpg
Completed Image ====> 16.mellowyellowyogi-960x640.jpg
URLError on an image...trying next one... Error: HT

In [10]:
!zip -r file.zip downloads

  adding: downloads/ (stored 0%)
  adding: downloads/padangusthasana/ (stored 0%)
  adding: downloads/padangusthasana/9-0.png (deflated 10%)
  adding: downloads/padangusthasana/46-0.png (deflated 1%)
  adding: downloads/padangusthasana/94-0.png (deflated 6%)
  adding: downloads/padangusthasana/10-0.png (deflated 4%)
  adding: downloads/padangusthasana/29-1.png (deflated 2%)
  adding: downloads/padangusthasana/34-0.png (deflated 8%)
  adding: downloads/padangusthasana/51-0.png (deflated 1%)
  adding: downloads/padangusthasana/36-3.png (deflated 1%)
  adding: downloads/padangusthasana/47-0.png (deflated 1%)
  adding: downloads/padangusthasana/7-1.png (deflated 4%)
  adding: downloads/padangusthasana/44-1.png (deflated 6%)
  adding: downloads/padangusthasana/50-0.png (deflated 5%)
  adding: downloads/padangusthasana/69-1.png (deflated 1%)
  adding: downloads/padangusthasana/92-0.png (deflated 2%)
  adding: downloads/padangusthasana/36-0.png (deflated 2%)
  adding: downloads/padangusthasan